In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install yt-dlp
!pip install gradio

import gradio as gr
import subprocess
import os

def download_and_transcribe(urls):
    messages = []  # Collect messages to update the interface
    urls = urls.strip().split('\n')  # Split input into separate URLs
    audio_counter = 1  # Counter to name audio files sequentially
    for url in urls:
        if url.strip():  # Ensure the URL is not just empty spaces
            audio_filename = f"audio{audio_counter}"
            messages.append(f"Downloading audio from: {url} as {audio_filename}")

            # Download audio
            try:
                yt_dlp_command = f"yt-dlp -x -o {audio_filename} --force-overwrites {url}"
                subprocess.run(yt_dlp_command, shell=True, check=True)

                messages.append(f"Transcribing audio: {audio_filename}")

                # Run whisper
                whisper_command = f"whisper {audio_filename}.opus --model large"
                result = subprocess.run(whisper_command, shell=True, capture_output=True, text=True)

                # Save result to file
                with open(f"transcription{audio_counter}.txt", "w") as f:
                    f.write(result.stdout)

                messages.append(f"Transcription completed: transcription{audio_counter}.txt")

                audio_counter += 1  # Increment counter for the next file
            except Exception as e:
                messages.append(f"Failed to process {url}: {str(e)}")
            finally:
                # Output messages for each step
                yield "\n".join(messages)

def interface_function(urls):
    return "\n".join(download_and_transcribe(urls))

with gr.Blocks() as app:
    with gr.Row():
        text_area = gr.Textbox(lines=7, placeholder="Enter YouTube URLs separated by new lines", label="YouTube URLs")
        button = gr.Button("Process URLs")
    output_text = gr.Textbox(label="Process Output", interactive=True, lines=10)

    button.click(
        fn=interface_function,
        inputs=text_area,
        outputs=output_text
    )

app.launch()
